In [1]:
import networkx as nx
import EoN
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from contact_tracing import *

import math
from constraint import *
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ortools.linear_solver import pywraplp
from ortools.linear_solver.pywraplp import Constraint, Solver, Variable, Objective

from solve import *

from typing import Set, Dict
import pickle as pkl

In [2]:
G = grab_graph() #fetching the montgamery graph
n = len(G.nodes)
I0 = [i for i in range(n) if random.random() > 0.999]

In [3]:
COSTS = np.ones(n)

In [4]:
(V_1,V_2) = find_contours(G,I0)

In [5]:
P,Q = PQ(G,I0,runs = 200)

In [6]:
print(len(Q['u'].unique()))
print(len(Q['v'].unique()))

print(len(V_1))
print(len(V_2))

1425
35710
1446
34771


In [7]:
prob = ProbMinExposed.from_dataframe(G,I0,V_1,V_2,P,Q,10,COSTS)

p1: 0.4650000000000003
p1: 0.5200000000000004
p1: 0.5050000000000003
p1: 0.5100000000000003
p1: 0.7550000000000006
p1: 0.5550000000000004
p1: 0.49500000000000033
p1: 0.5300000000000004
p1: 0.4700000000000003
p1: 0.5050000000000003
p1: 0.5550000000000004
p1: 0.5600000000000004
p1: 0.4800000000000003
p1: 0.5650000000000004
p1: 0.49500000000000033
p1: 0.49500000000000033
p1: 0.5050000000000003
p1: 0.5500000000000004
p1: 0.5000000000000003
p1: 0.5300000000000004
p1: 0.4850000000000003
p1: 0.4850000000000003
p1: 0.4800000000000003
p1: 0.5000000000000003
p1: 0.42500000000000027
p1: 0.5200000000000004
p1: 0.5400000000000004
p1: 0.5350000000000004
p1: 0.4750000000000003
p1: 0.4850000000000003
p1: 0.4700000000000003
p1: 0.5150000000000003
p1: 0.7550000000000006
p1: 0.4700000000000003
p1: 0.5300000000000004
p1: 0.5050000000000003
p1: 0.5100000000000003
p1: 0.5000000000000003
p1: 0.5200000000000004
p1: 0.5100000000000003
p1: 0.7450000000000006
p1: 0.4750000000000003
p1: 0.5100000000000003
p1: 0.4

In [ ]:
#p1 = defaultdict(lambda: 1)
#q = defaultdict(lambda: defaultdict(lambda : 1))
#prob = ProbMinExposed(G,I0,V_1,V_2,p1,q,10,COSTS)

In [ ]:
prob.solve_lp()
probabilities = np.array(prob.getVariables())

In [ ]:
len(probabilities)

In [8]:
a = optimized_iterated_round(prob, 200)

In [10]:
a.sum()

10.0

In [ ]:
"""STOP HERE"""

In [ ]:
for u in V1:
    for v in G.neighbors(u):
        if v in V2:
            print(u)
            print(v)
            #coeff = (q[u][v] * self.p1[u])
            #print(f"Coeff: {coeff}")
            #self.solver.Add(self.Y2[v] >= coeff * self.Y1[u])

In [ ]:
G.adj[1076]

In [ ]:
Q[Q.u == 1076]

In [ ]:
prob.getVariables()

In [ ]:
#test simplify

a = 1
b = 0

total_a = 0
total_b = 0

n=1000000

for i in range(n):
    (x,y,new_a,new_b) = simplify(a,b)
    
    if x==-1:
        total_a += new_a
        total_b += y
    elif y==-1:
        total_a += x
        total_b += new_b
    else:
        total_a += x
        total_b += y

print(str(total_a/n) + " " + str(a))
print(str(total_b/n) + " " + str(b))

In [ ]:
#test D
p = [.5,.5,.25,.6,.15]

n=10000
total = np.zeros(5)

for i in range(n):
    a = D(p)
    if np.sum(a) != np.sum(p):
        print("oof")
    #print(p)
    total += a
    
total/=n
print(total)

In [ ]:
#test D_prime
p = [.5,.4,.25,.75,.7]

n=5000
total = np.zeros(5)

for i in range(n):
    a = D(p)
    if (np.sum(a) != math.floor(np.sum(p))) & (np.sum(a) != math.ceil(np.sum(p))):
        print("oof")
    #print(p)
    total += a
    
total/=n
print(total)